In [4]:
import os
import pandas as pd

# 指定主文件夹路径
base_dir = "Archive 2020-2023"

# 目标年份
years_to_merge = ["2020", "2021", "2022", "2023"]

# 存储所有 DataFrame
dfs = []

# 遍历指定年份的文件夹
for year in years_to_merge:
    year_path = os.path.join(base_dir, year)
    
    if os.path.exists(year_path):
        # 遍历该年份文件夹内的所有 CSV 文件
        for file in os.listdir(year_path):
            if file.endswith(".csv"):
                file_path = os.path.join(year_path, file)
                df = pd.read_csv(file_path)  # 读取 CSV 文件
                dfs.append(df)  # 添加到列表中

# 合并所有 DataFrame
if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)
    # 打印合并后的数据
    print(merged_df)
else:
    print("未找到任何 CSV 文件")

/var/folders/6k/6gh7mwnd44g1q3ngnf39sklh0000gn/T/ipykernel_9472/3200449878.py:22: DtypeWarning: Columns (13,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # 读取 CSV 文件
/var/folders/6k/6gh7mwnd44g1q3ngnf39sklh0000gn/T/ipykernel_9472/3200449878.py:22: DtypeWarning: Columns (13,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # 读取 CSV 文件
/var/folders/6k/6gh7mwnd44g1q3ngnf39sklh0000gn/T/ipykernel_9472/3200449878.py:22: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # 读取 CSV 文件
/var/folders/6k/6gh7mwnd44g1q3ngnf39sklh0000gn/T/ipykernel_9472/3200449878.py:22: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # 读取 CSV 文件
/var/folders/6k/6gh7mwnd44g1q3ngnf39sklh0000gn/T/ipykernel_9472/3200449878.py:22: DtypeWar

         alert_id          cause   cause_detail        effect effect_detail  \
0          338650  UNKNOWN_CAUSE  UNKNOWN_CAUSE  OTHER_EFFECT  TRACK_CHANGE   
1          338650  UNKNOWN_CAUSE  UNKNOWN_CAUSE  OTHER_EFFECT  TRACK_CHANGE   
2          338650  UNKNOWN_CAUSE  UNKNOWN_CAUSE  OTHER_EFFECT  TRACK_CHANGE   
3          338650  UNKNOWN_CAUSE  UNKNOWN_CAUSE  OTHER_EFFECT  TRACK_CHANGE   
4          338650  UNKNOWN_CAUSE  UNKNOWN_CAUSE  OTHER_EFFECT  TRACK_CHANGE   
...           ...            ...            ...           ...           ...   
1890032    507845  UNKNOWN_CAUSE  UNKNOWN_CAUSE  OTHER_EFFECT         DELAY   
1890033    507846    OTHER_CAUSE        TRAFFIC  OTHER_EFFECT         DELAY   
1890034    507848  UNKNOWN_CAUSE  UNKNOWN_CAUSE  OTHER_EFFECT         DELAY   
1890035    507848  UNKNOWN_CAUSE  UNKNOWN_CAUSE  OTHER_EFFECT         DELAY   
1890036    507849    OTHER_CAUSE        TRAFFIC  OTHER_EFFECT         DELAY   

        severity_level  severity  \
0              

In [18]:
import pandas as pd

# 假设你的 DataFrame 已加载为 merged_df
# 1. 筛选出包含 "delay" 的行，并且 route_id 在指定的路由 ID 列表中
route_ids = ["Green-B", "Green-C", "Green-D", "Green-E", "Red", "Blue", "Orange", "Mattapan"]
delay_df = merged_df[
    merged_df['header'].str.contains('delay', case=False, na=False) &
    merged_df['route_id'].isin(route_ids)
]

# 2. 将时间列转换为 datetime 格式
delay_df['created_dt'] = pd.to_datetime(delay_df['created_dt'])
delay_df['last_modified_dt'] = pd.to_datetime(delay_df['last_modified_dt'])

# 3. 计算延迟时间
delay_df['delay_duration'] = (delay_df['last_modified_dt'] - delay_df['created_dt']).dt.total_seconds()

# 4. 添加 year 列，提取 created_dt 中的年份
delay_df['year'] = delay_df['created_dt'].dt.year

# 5. 加载 MBTA_Rapid_Transit_Stop_Distances.csv 文件
stop_distances_df = pd.read_csv('MBTA_Rapid_Transit_Stop_Distances.csv')

# 6. 获取 stop_name，通过 stop_id 和 from_stop_id 匹配
stop_distances_df = stop_distances_df[['from_stop_id', 'from_stop_name']].rename(columns={'from_stop_id': 'stop_id', 'from_stop_name': 'stop_name'})

# 7. 将 stop_id 列转换为字符串类型，以便于匹配
delay_df['stop_id'] = delay_df['stop_id'].astype(str)
stop_distances_df['stop_id'] = stop_distances_df['stop_id'].astype(str)

# 8. 将 stop_name 添加到 delay_df 中，通过 stop_id 匹配
delay_df = delay_df.merge(stop_distances_df, on='stop_id', how='left')

# 9. 按照 route_id、stop_name 和 year 分组，计算每个组的延迟时间总和
grouped_delay = delay_df.groupby(['route_id', 'stop_name', 'year'])['delay_duration'].sum().reset_index()

# 10. 打印结果
print(grouped_delay)

/var/folders/6k/6gh7mwnd44g1q3ngnf39sklh0000gn/T/ipykernel_9472/3412671004.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delay_df['created_dt'] = pd.to_datetime(delay_df['created_dt'])
/var/folders/6k/6gh7mwnd44g1q3ngnf39sklh0000gn/T/ipykernel_9472/3412671004.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delay_df['last_modified_dt'] = pd.to_datetime(delay_df['last_modified_dt'])
/var/folders/6k/6gh7mwnd44g1q3ngnf39sklh0000gn/T/ipykernel_9472/3412671004.py:16: SettingWithCopyWarning: 
A value i

     route_id      stop_name  year  delay_duration
0        Blue        Airport  2020        283428.0
1        Blue        Airport  2021         36345.0
2        Blue        Airport  2022       5729123.0
3        Blue        Airport  2023         83922.0
4        Blue       Aquarium  2020          1192.0
...       ...            ...   ...             ...
1026      Red  South Station  2023       1093320.0
1027      Red      Wollaston  2020        293634.0
1028      Red      Wollaston  2021        206976.0
1029      Red      Wollaston  2022        528888.0
1030      Red      Wollaston  2023        380991.0

[1031 rows x 4 columns]


In [21]:
# Assuming you have the grouped_delay DataFrame ready
grouped_delay.to_csv('grouped_delay.csv', index=False)